In [ ]:
# Tarea3-Jose Maria Saldaño Guijarro

## Configuración

He creado un repositorio llamado  Tarea3-SM y lo he clonado en local con el comando "git clone <url>" .
Después he creado un entorno con el comando "conda create --name=tarea3" y lo activo con el comando "conda activate tarea3".
Instalo Python 3.10, ipykernel y JupyterLab en el entorno y añado el entorno conda al kernel de JupyterLab con el comando "python3 -m ipykernel install --user --name=tarea3".
Instalo JupyterLab en el entorno tarea3 y ejecuto JupyterLab y creo un notebook vacío seleccionando como kernel tarea3.



In [ ]:
# Importacion.
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt

## He tenido que cambiar el kernel porque tras instalar spicy seguía sin funcionar.

### Especificamos directorios de entrada y salida

In [ ]:
# Directorios que usaremos.
cwd = os.getcwd()
audio_input_path = os.path.join(cwd, os.path.join('audio', 'examples'))  
audio_output_path = os.path.join(cwd, os.path.join('audio', '_output'))
print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')


In [ ]:
### Cargamos audio

In [ ]:
# Cargamos el archivo de audio.
filename = os.path.join(audio_input_path, 'breaking_bad.wav')
# audio_data, sample_rate = librosa.load(filename, sr=None, mono=False)
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

### Probamos el audio

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate)

### Mostramos características

In [ ]:
# Mostrar informacion del tamaño , canales, resolución (sonido estéreo).
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}')
print(f'- 1º canal:   {audio_data[:5, 0]}...')
print(f'- 2º canal:   {audio_data[:5, 1]}...')
print(f'- Resolucion: {type(audio_data[0,0])}\n')

### Convertimos de estéreo a mono y mostramos las características 

In [ ]:
# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Column-wise.
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')
print(f'- Canal unico:  {new_data_mono[:5]}...')

# Mantenemos la misma resolucion que antes.
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')


### Guardamos el archivo mono


In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, 'sample1_mono.wav'),
    rate=sample_rate,
    data=new_data_mono
)


In [ ]:
### Probamos el audio mono

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)


### Diferencia entre mono y estéreo

La gran diferencia es que en mono a la hora de grabar y reproducir sólo existe un único canal y en estéreo tenemos dos canales y no se reproduce lo mismo por los dos canales, existen diferencia entre los dos canales.

### Comprobamos la diferencia de tamaño en cada archivo

In [ ]:
!ls -sh audio/examples/breaking_bad.wav
!ls -sh audio/_output/sample1_mono.wav

In [ ]:
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz\n')

## Segunda parte de la práctica

### Gráfica en el dominio del tiempo para el audio mono y estéreo

In [ ]:
#cargamos un nuevo auido estereo

sample_rate_emotional, audio_data_emotional = wavfile.read(filename=os.path.join(audio_input_path, 'emotional_soundtrack.wav'))



In [ ]:
ampl_values_mono = len(new_data_mono)
ampl_values_emotional = len(audio_data_emotional)
print(f'Número de muestras del audio viola (valores de amplitud): {ampl_values_mono}')
print(f'Número de muestras del audio emotional (valores de amplitud): {ampl_values_emotional}')

In [ ]:
# Construimos el array para el eje x que representa el tiempo de la grabación.
# Tiene la forma: np.arange(Vi, Vf, P). 
t1 = np.arange(0, ampl_values_mono/sample_rate, 1/sample_rate)
t2 = np.arange(0, ampl_values_emotional/sample_rate_emotional, 1/sample_rate_emotional)


In [ ]:
print(t1)
print(t2)

In [ ]:
# Creamos la figura.
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Solo mostramos las primeras 50 muestras de amplitud (por claridad).
end = 50

# Señal mono.
ax[0].plot(t1[:end], new_data_mono[:end], marker='X')
ax[0].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate} Hz')
ax[0].set_ylabel('Amplitud')
ax[0].grid(True)

# Señal emotional.
# Utilizamos ratio para ajustar el eje x de ambas gráficas
# ya que la fs es menor en esta señal.
ratio = sample_rate / sample_rate_emotional  
ax[1].plot(t2[:int(end/ratio)], audio_data_emotional[:int(end/ratio)], c='tab:red', marker='X')
ax[1].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate_emotional} Hz')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Amplitud')
ax[1].grid(True)

# Mostramos la figura.
plt.tight_layout()
plt.show()

### Frecuencia de muestreo

Es el número de muestras que tomamos por unidad de tiempo de una señal analógica para conventirla en digital.

### Aliasing

Este efecto sucede cuando diferentes señales continuas al transformarlas a digitales no se pueden distinguir debido a una frecuencia de muestreo muy baja.

### Profundidad de bits

Es la cantidad de bits utilizado para medir una onda sonora.

### Ancho de banda

El ancho de banda es la longitud de la extensión de frecuencias en la que se concentra la mayor potencia de la señal. Se puede calcular con la transformada de Fourier.

### Tasa de bits

Es el número de bits que se transmiten por segundo a través de un sistema de transmisión digital o entre  dispositivos digitales.

## Transformada de Fourier de audio mono

In [ ]:
# La longitud del array de datos y el
# sample rate (frecuencia de muestreo).
n = len(new_data_mono)
Fs = sample_rate


# Calculando la Transformada Rapida de Fourier (FFT) en audio mono.
ch_Fourier = np.fft.fft(new_data_mono)  # ch1

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

La "Transformación rápida de Fourier"  descompone una señal en sus componentes espectrales individuales, proporcionando información sobre su composición.

### Energia del espectrograma y frecuencia de corte

Nos vamos a quedar con las frencuencias que estén por debajo de la frecuencia umbral, este valor lo definiremos nosotros.

Vamos a probar con varios epsilons y vamos a comprobar que a medida que aumenta su valor, se mantienen menos frecuencias por lo que la calidad del audio es peor aunque el tamaño del archivo de salida sea más reducido,

In [ ]:
# Definimos diferentes epsilons: la parte de
# la energia del espectro que NO conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (ID VARIANDO ESTE VALOR Y MIRAD LA GRÁFICA).
eps = eps[0]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el
# valor de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

Cambiamos el epsilon

In [ ]:
# Definimos diferentes epsilons: la parte de
# la energia del espectro que NO conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (ID VARIANDO ESTE VALOR Y MIRAD LA GRÁFICA).
eps = eps[3]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el
# valor de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

Probamos con el mayor epsilon definido

In [ ]:
# Definimos diferentes epsilons: la parte de
# la energia del espectro que NO conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (ID VARIANDO ESTE VALOR Y MIRAD LA GRÁFICA).
eps = eps[6]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el
# valor de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

Vemos que mientras mayor es el epsilon menor es la frecuencia de corte.

### Comprimir la onda aplicando downsampling

In [ ]:
# Definimos los nombres de los audios comprimidos.
wav_compressed_file = "sample_mono_compressed.wav"

# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = new_data_mono[::D]

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(audio_output_path, wav_compressed_file))

Lo escuchamos para ver la diferencia de calidad

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

Vemos que el tamaño final es menor

In [ ]:
!ls -sh audio/_output/sample_mono_compressed.wav

### Analizamos el espectograma de ambas ondas

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(new_data_mono, NFFT=1024, Fs=sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()


La primera gran diferencia que visualizo son los cambios en la frecuencia, en el audio original las frecuencias son mucho más altas que las del audio comprimido, esto era de esperar debido al factor de downsampling que hemos calculado con la frecuencia de corte.

Otra diferencia es que en el espectograma del audio comprimido aparecen algunas distorsiones que en el audio original no existen.

### Mostramos el tamaño de ambos archivos

In [ ]:
!ls -sh audio/_output/sample1_mono.wav
!ls -sh audio/_output/sample_mono_compressed.wav

### Reproducción de ambos archivos

In [ ]:
#Archivo original
IPython.display.Audio(new_data_mono, rate=sample_rate)



In [ ]:
#Archivo comprimido
IPython.display.Audio(new_audio_data, rate=new_sample_rate)